In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from wordcloud import WordCloud

import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence #unique id

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout, Embedding
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df=pd.read_csv('/content/Stress.csv')

In [ ]:
df.head()

,subreddit,post_id,sentence_range,text,label,confidence,social_timestamp
0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",1,0.8,1521614353
1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",0,1.0,1527009817
2,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,1,0.8,1535935605
3,relationships,7rorpp,"[5, 10]","until i met my new boyfriend, he is amazing, h...",1,0.6,1516429555
4,survivorsofabuse,9p2gbc,"[0, 5]",October is Domestic Violence Awareness Month a...,1,0.8,1539809005


In [ ]:
df.drop(columns=["subreddit","post_id","sentence_range","confidence","social_timestamp"],inplace=True)

In [ ]:
df.head()

,text,label
0,"He said he had not felt that way before, sugge...",1
1,"Hey there r/assistance, Not sure if this is th...",0
2,My mom then hit me with the newspaper and it s...,1
3,"until i met my new boyfriend, he is amazing, h...",1
4,October is Domestic Violence Awareness Month a...,1


In [ ]:
df.isnull().sum()

text     0
label    0
dtype: int64

In [ ]:
def cleantext(text):
  tokens = word_tokenize(text.lower())
  ftoken = [t for t in tokens if(t.isalpha())]
  stop = stopwords.words("english")
  ctoken = [t for t in ftoken if(t not in stop)]
  lemma = WordNetLemmatizer()
  ltoken = [lemma.lemmatize(t) for t in ctoken]
  return " ".join(ltoken)

In [ ]:
df["label"].value_counts()

1    1488
0    1350
Name: label, dtype: int64

In [ ]:
df["text"] =df["text"].apply(cleantext)

In [ ]:
df.head()

,text,label
0,said felt way suggeted go rest trigger ahead y...,1
1,hey sure right place post go currently student...,0
2,mom hit newspaper shocked would know like play...,1
3,met new boyfriend amazing kind sweet good stud...,1
4,october domestic violence awareness month dome...,1


In [ ]:
x = df["text"]
y = df["label"]

In [ ]:
sentlen = []

for sent in df["text"]:
  sentlen.append(len(word_tokenize(sent)))

df["SentLen"] = sentlen 
df.head()

,text,label,SentLen
0,said felt way suggeted go rest trigger ahead y...,1,51
1,hey sure right place post go currently student...,0,55
2,mom hit newspaper shocked would know like play...,1,67
3,met new boyfriend amazing kind sweet good stud...,1,111
4,october domestic violence awareness month dome...,1,45


In [ ]:
max_len = np.quantile(sentlen, 0.95)

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=1)

In [ ]:
tok = Tokenizer(char_level=False, split=" ")
tok.fit_on_texts(xtrain)
tok.index_word

{1: 'like',
 2: 'time',
 3: 'get',
 4: 'know',
 5: 'feel',
 6: 'would',
 7: 'year',
 8: 'really',
 9: 'want',
 10: 'even',
 11: 'thing',
 12: 'help',
 13: 'one',
 14: 'go',
 15: 'day',
 16: 'friend',
 17: 'people',
 18: 'could',
 19: 'make',
 20: 'work',
 21: 'back',
 22: 'think',
 23: 'going',
 24: 'life',
 25: 'anxiety',
 26: 'much',
 27: 'never',
 28: 'month',
 29: 'got',
 30: 'also',
 31: 'need',
 32: 'still',
 33: 'something',
 34: 'job',
 35: 'way',
 36: 'take',
 37: 'told',
 38: 'family',
 39: 'see',
 40: 'relationship',
 41: 'anyone',
 42: 'first',
 43: 'say',
 44: 'ca',
 45: 'thought',
 46: 'feeling',
 47: 'since',
 48: 'anything',
 49: 'said',
 50: 'lot',
 51: 'week',
 52: 'good',
 53: 'home',
 54: 'last',
 55: 'right',
 56: 'well',
 57: 'always',
 58: 'someone',
 59: 'love',
 60: 'started',
 61: 'person',
 62: 'every',
 63: 'talk',
 64: 'bad',
 65: 'night',
 66: 'trying',
 67: 'getting',
 68: 'come',
 69: 'u',
 70: 'made',
 71: 'tell',
 72: 'around',
 73: 'long',
 74: 'went'

In [ ]:
vocab_len = len(tok.index_word)
vocab_len

8236

In [ ]:
seqtrain = tok.texts_to_sequences(xtrain) #step1
seqtrain

[[260,
  639,
  73,
  116,
  78,
  4466,
  489,
  222,
  235,
  2610,
  770,
  1039,
  588,
  160,
  360,
  9,
  129,
  106,
  2206,
  419],
 [16,
  4467,
  129,
  260,
  229,
  640,
  21,
  2611,
  441,
  384,
  1267,
  1180,
  100,
  4,
  59,
  32,
  4,
  48,
  9,
  32,
  15,
  107,
  384,
  57,
  130,
  46,
  129,
  95,
  1040,
  565,
  1702,
  40,
  174,
  169,
  223,
  879,
  835,
  805,
  335,
  7,
  27,
  3,
  21,
  79,
  1927],
 [1525,
  32,
  24,
  524,
  109,
  85,
  204,
  73,
  224,
  267,
  131,
  524,
  109,
  268,
  836,
  196,
  137,
  372,
  340,
  546,
  16,
  3,
  372,
  1,
  4468,
  17,
  91,
  806,
  166,
  45,
  734,
  693,
  37,
  524,
  109,
  268,
  67,
  327,
  114,
  269,
  589,
  431],
 [348,
  76,
  4,
  115,
  1041,
  4469,
  1928,
  2612,
  2613,
  604,
  54,
  51,
  5,
  1,
  385,
  341,
  83,
  83,
  2614,
  76,
  270,
  36,
  3273,
  1929,
  27,
  3274,
  55,
  75,
  386,
  354,
  668,
  161,
  1042,
  314,
  735,
  186,
  90,
  4,
  41,
  116],
 [6,
 

In [ ]:
seqmattrain = sequence.pad_sequences(seqtrain, maxlen= int(max_len)) #step2
seqmattrain

array([[   0,    0,    0, ...,  106, 2206,  419],
       [   0,    0,    0, ...,   21,   79, 1927],
       [   0,    0,    0, ...,  269,  589,  431],
       ...,
       [   0,    0,    0, ...,  805, 1003,  874],
       [   0,    0,    0, ...,   83,  161,  235],
       [   0,    0,    0, ...,  490, 8236,  144]], dtype=int32)

In [ ]:
seqtest = tok.texts_to_sequences(xtest)
seqmattest = sequence.pad_sequences(seqtest, maxlen=int(max_len))

In [ ]:
vocab_len

8236

In [ ]:
rnn = Sequential()

rnn.add(Embedding(vocab_len+1,100, input_length=int(max_len), mask_zero=True))
rnn.add(SimpleRNN(units=32, activation="tanh"))
rnn.add(Dense(units=32, activation="relu"))
rnn.add(Dropout(0.2))

rnn.add(Dense(units=1, activation="sigmoid"))

rnn.compile(optimizer="adam", loss="binary_crossentropy")

rnn.fit(seqmattrain, ytrain, batch_size=50, epochs=25)

ypred = rnn.predict(seqmattest)

ypred = ypred>0.5

Epoch 1/25
40/40 [==============================] - 5s 45ms/step - loss: 0.6853
Epoch 2/25
40/40 [==============================] - 2s 45ms/step - loss: 0.4565
Epoch 3/25
40/40 [==============================] - 2s 46ms/step - loss: 0.1448
Epoch 4/25
40/40 [==============================] - 2s 48ms/step - loss: 0.0372
Epoch 5/25
40/40 [==============================] - 2s 46ms/step - loss: 0.0112
Epoch 6/25
40/40 [==============================] - 3s 66ms/step - loss: 0.0064
Epoch 7/25
40/40 [==============================] - 2s 49ms/step - loss: 0.0066
Epoch 8/25
40/40 [==============================] - 2s 45ms/step - loss: 0.0045
Epoch 9/25
40/40 [==============================] - 2s 45ms/step - loss: 0.0044
Epoch 10/25
40/40 [==============================] - 2s 46ms/step - loss: 0.0039
Epoch 11/25
40/40 [==============================] - 2s 45ms/step - loss: 0.0033
Epoch 12/25
40/40 [==============================] - 2s 53ms/step - loss: 0.0031
Epoch 13/25
40/40 [==================

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ytest,ypred))

              precision    recall  f1-score   support

           0       0.65      0.57      0.61       394
           1       0.67      0.74      0.70       458

    accuracy                           0.66       852
   macro avg       0.66      0.65      0.65       852
weighted avg       0.66      0.66      0.66       852

